In [1]:
from s3dis_dataset import S3DISValidator
from plane_detector import StructuralRANSAC
# from ransac_numpy import StructuralRANSAC
import matplotlib.pyplot as plt
import open3d as o3d
import numpy as np
# from pointnet_utils import inference, create_pointnet_blocks, reconstruct_furniture_labels
from s3dis_metrics import SegmentationMetrics
import wandb
import yaml

In [3]:
C = ["ceiling", "floor", "wall", "window", "door"]
metrics = SegmentationMetrics(C)

In [11]:
{k: v + 1 for k, v in metrics.results.items()}

{'ceiling': 1.0, 'floor': 1.0, 'wall': 1.0, 'window': 1.0, 'door': 1.0}

In [2]:
with open('config.yaml') as f:
    config = yaml.safe_load(f)

In [3]:
config

{'architecture': 'RANSAC + PointNet++',
 'dataset': 's3dis',
 'area': 5,
 'room': 'office_5',
 'subsample_method': 'voxel',
 'block_size': 1.0,
 'stride': 0.5,
 'wall_angle_tolerance': 30}

In [4]:
s_val = S3DISValidator("../s3DIS/Stanford3dDataset_v1.2_Aligned_Version/")

In [5]:
points, gt_labels = s_val.load_and_process_room(area=config["area"], room=config["room"], subsample_method=config["subsample_method"])

Loading ../s3DIS/Stanford3dDataset_v1.2_Aligned_Version//Area_5/office_5 with voxel size 0.02m
  Original points: 766,453
  Subsampled points: 194,199 (25.3%)


In [6]:
xyz = points[:, :3]
colors = points[:, 3:]

In [7]:
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(xyz)
pcd.colors = o3d.utility.Vector3dVector(colors)

In [8]:
# o3d.visualization.draw_geometries([pcd])

In [9]:
segmenter = StructuralRANSAC(pcd, downsample=False)
segments = segmenter.segment()

Starting structural segmentation...
Detecting floor and ceiling...
  Floor found: 28538 points
  Ceiling found: 35516 points
Detecting walls with validation...
  Wall 0 found: 29083 points
  Wall 1 found: 16147 points
  Wall 2 found: 14959 points
  Total: 3 walls, 0 furniture pieces
Segmentation complete in 30.78 seconds
Found: 5 structural elements
  floor: 1
  ceiling: 1
  wall: 3
Remaining unsegmented points: 69956


In [10]:
segmenter.visualize_segments()

PointCloud with 194199 points.

In [11]:
point_labels = map_ransac_to_s3dis(segments, gt_labels)
metrics = SegmentationMetrics(gt_labels, point_labels, s_val.class_map.keys())
results = metrics.compute_metrics()
print(f"Overall Accuracy: {results['overall_acc']:.2%}")

print("\nPer-class IoU:")
for class_name, iou in results['iou_per_class'].items():
    print(f"  {class_name}: {iou:.2%}")

Overall Accuracy: 55.75%

Per-class IoU:
  ceiling: 93.34%
  floor: 93.40%
  wall: 69.49%
  beam: 0.00%
  column: 0.00%
  window: 0.00%
  door: 0.00%
  table: 0.00%
  chair: 0.00%
  sofa: 0.00%
  bookcase: 0.00%
  board: 0.00%
  clutter: 0.00%


In [37]:
seg_dict = segments.copy()

In [13]:
for segment in seg_dict:
    if segment.startswith("wall"):
        print(segment)
        idx = seg_dict[segment]["indices"]
        print(len(idx))
        wall_xyz = xyz[idx]
        wall_colors = colors[idx]
        wall_points = np.concatenate([wall_xyz, wall_colors], axis=1)
        data_blocks, block_indices = create_pointnet_blocks(wall_points, config["block_size"], config["stride"])
        all_predictions, all_confidences = inference(data_blocks)
        furniture_labels = reconstruct_furniture_labels(
            wall_points, all_predictions, all_confidences, block_indices
        )
        
        refined = segmenter.refine_wall_segment(segment, idx, furniture_labels)

wall_0
29083
Processing furniture block 1/10
Processing furniture block 6/10
added board
added bookcase
wall_1
16147
Processing furniture block 1/7
Processing furniture block 6/7
added door
added board
added bookcase
wall_2
14959
Processing furniture block 1/9
Processing furniture block 6/9
added beam
added door
added bookcase


In [14]:
segmenter.visualize_segments()

PointCloud with 194199 points.

In [15]:
point_labels = map_ransac_to_s3dis(segments, gt_labels)
metrics = SegmentationMetrics(gt_labels, point_labels, s_val.class_map.keys())
results = metrics.compute_metrics()
print(f"Overall Accuracy: {results['overall_acc']:.2%}")

print("\nPer-class IoU:")
for class_name, iou in results['iou_per_class'].items():
    print(f"  {class_name}: {iou:.2%}")

Overall Accuracy: 55.05%

Per-class IoU:
  ceiling: 93.34%
  floor: 93.40%
  wall: 66.31%
  beam: 0.00%
  column: 0.00%
  window: 0.00%
  door: 15.98%
  table: 0.00%
  chair: 0.00%
  sofa: 0.00%
  bookcase: 16.41%
  board: 0.00%
  clutter: 0.00%


In [18]:
to_remove = []
for segment in segments:
    to_remove.extend(segments[segment]["indices"])
remaining_indices = np.arange(len(xyz))
remaining_indices = np.setdiff1d(remaining_indices, to_remove)

In [1]:
r = pcd.select_by_index(remaining_indices)

NameError: name 'pcd' is not defined

In [ ]:
# o3d.visualization.draw_geometries([r])

In [20]:
rest_xyz = np.array(r.points)
rest_colors = np.array(r.colors)
rest_points = np.concatenate([rest_xyz, rest_colors], axis=1)
data_blocks, block_indices = create_pointnet_blocks(rest_points, config["block_size"], config["stride"])
all_predictions, all_confidences = inference(data_blocks)
furniture_labels = reconstruct_furniture_labels(
    rest_points, all_predictions, all_confidences, block_indices
)


Processing furniture block 1/69
Processing furniture block 6/69
Processing furniture block 11/69
Processing furniture block 16/69
Processing furniture block 21/69
Processing furniture block 26/69
Processing furniture block 31/69
Processing furniture block 36/69
Processing furniture block 41/69
Processing furniture block 46/69
Processing furniture block 51/69
Processing furniture block 56/69
Processing furniture block 61/69
Processing furniture block 66/69


In [33]:
wall_mask = furniture_labels == 2
beam_mask = furniture_labels == 3
column_mask = furniture_labels == 4
window_mask = furniture_labels == 5
door_mask = furniture_labels == 6
table_mask = furniture_labels == 7
chair_mask = furniture_labels == 8
sofa_mask = furniture_labels == 9
bookcase_mask = furniture_labels == 10

In [35]:
segments["wall"] = {
                "indices": remaining_indices[wall_mask],
                "type": "wall",
                "parent": "rest",
            }

In [22]:
segments["beam"] = {
                "indices": remaining_indices[beam_mask],
                "type": "beam",
                "parent": "rest",
            }

In [23]:
segments["column"] = {
                "indices": remaining_indices[column_mask],
                "type": "column",
                "parent": "rest",
            }

In [34]:
segments["window"] = {
                "indices": remaining_indices[window_mask],
                "type": "window",
                "parent": "rest",
            }

In [24]:
segments["door"] = {
                "indices": remaining_indices[door_mask],
                "type": "door",
                "parent": "rest",
            }

In [25]:
segments["table"] = {
                "indices": remaining_indices[table_mask],
                "type": "table",
                "parent": "rest",
            }

In [26]:
segments["chair"] = {
                "indices": remaining_indices[chair_mask],
                "type": "chair",
                "parent": "rest",
            }

In [27]:
segments["sofa"] = {
                "indices": remaining_indices[sofa_mask],
                "type": "sofa",
                "parent": "rest",
            }

In [28]:
segments["bookcase"] = {
                "indices": remaining_indices[bookcase_mask],
                "type": "bookcase",
                "parent": "rest",
            }

In [36]:
point_labels = map_ransac_to_s3dis(segments, gt_labels)
metrics = SegmentationMetrics(gt_labels, point_labels, s_val.class_map.keys())
results = metrics.compute_metrics()
print(f"Overall Accuracy: {results['overall_acc']:.2%}")

print("\nPer-class IoU:")
for class_name, iou in results['iou_per_class'].items():
    print(f"  {class_name}: {iou:.2%}")

Overall Accuracy: 80.33%

Per-class IoU:
  ceiling: 93.34%
  floor: 93.40%
  wall: 65.53%
  beam: 0.00%
  column: 0.00%
  window: 58.94%
  door: 25.13%
  table: 65.23%
  chair: 67.01%
  sofa: 0.00%
  bookcase: 68.16%
  board: 0.00%
  clutter: 0.00%


In [74]:
segmenter.visualize_segments()

PointCloud with 224535 points.

In [182]:
def get_element(name, coords, colors, segments):
    idx = segments[name]["indices"]
    xyz = coords[idx]
    rgb = colors[idx]
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(xyz)
    pcd.colors = o3d.utility.Vector3dVector(rgb)
    return pcd

In [188]:
p = get_element("wall_4", xyz, colors, segments)

In [189]:
o3d.visualization.draw_geometries([p])

In [21]:
token = "415dd2b149285b382a2e695e3023f569695b8398"
wandb.login(key=token)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /Users/dmytroivanov/.netrc
wandb: Currently logged in as: ivanovd to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [22]:
run = wandb.init(
    
    project="point-cloud-sem-seg",
    
    config=config,
)

In [17]:
point_labels = map_ransac_to_s3dis(segments, gt_labels)
metrics = SegmentationMetrics(gt_labels, point_labels, s_val.class_map.keys())
results = metrics.compute_metrics()
print(f"Overall Accuracy: {results['overall_acc']:.2%}")

print("\nPer-class IoU:")
for class_name, iou in results['iou_per_class'].items():
    print(f"  {class_name}: {iou:.2%}")

Overall Accuracy: 69.86%

Per-class IoU:
  ceiling: 89.49%
  floor: 89.34%
  wall: 72.17%
  beam: 59.35%
  column: 0.00%
  window: 67.05%
  door: 31.30%


In [ ]:
run.log({"overall_acc": results["overall_acc"]})
run.log(results["iou_per_class"])
run.finish()

In [29]:
to_remove = []
for segment in segments:
    to_remove.extend(segments[segment]["indices"])
remaining_indices = np.arange(len(xyz))
remaining_indices = np.setdiff1d(remaining_indices, to_remove)

In [30]:
r = pcd.select_by_index(remaining_indices)

In [31]:
o3d.visualization.draw_geometries([r])

In [83]:
segment_models = {}
segments_ransac = {}
segments_inliers = {}
epsilon = 0.15
min_cluster_points = 150
max_plane_idx = 10
distance_threshold=0.3
rest = r
for i in range(max_plane_idx):
    # colors = plt.get_cmap("tab20")(i)
    segment_models[i], inliers = rest.segment_plane(distance_threshold=distance_threshold, ransac_n=3, num_iterations=2000)
    segments_ransac[i] = rest.select_by_index(inliers)
    labels = np.array(segments_ransac[i].cluster_dbscan(eps=epsilon, min_points=min_cluster_points))
    candidates = [len(np.where(labels == j)[0]) for j in np.unique(labels)]
    best_candidate = int(np.unique(labels)[np.where(candidates == np.max(candidates))[0]])
    rest = rest.select_by_index(inliers, invert=True) + segments_ransac[i].select_by_index(list(np.where(labels != best_candidate)[0]))
    segments_ransac[i] = segments_ransac[i].select_by_index(list(np.where(labels == best_candidate)[0]))
    colors = plt.get_cmap("tab20")(i)
    segments_ransac[i].paint_uniform_color(list(colors[:3]))
    # segments_ransac[i].paint_uniform_color(list(colors[:3]))
    # rest = rest.select_by_index(inliers, invert=True)
    segments_inliers[i] = inliers
    print("pass", i, "/", max_plane_idx, "done.")

pass 0 / 10 done.
pass 1 / 10 done.
pass 2 / 10 done.
pass 3 / 10 done.
pass 4 / 10 done.
pass 5 / 10 done.
pass 6 / 10 done.
pass 7 / 10 done.
pass 8 / 10 done.
pass 9 / 10 done.


/var/folders/j4/yzpk21j93c5bl_mjst9qb4mc0000gn/T/ipykernel_33359/2417162630.py:15: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  best_candidate = int(np.unique(labels)[np.where(candidates == np.max(candidates))[0]])


In [88]:
def filter_wall_elements_by_normal(segments_ransac, segment_models):
    """
    Filter candidates based on plane normal orientation
    """

    y_candidates = []    # Potential elements along Y wall
    x_candidates = []    # Potential elements along X wall

    
    for i in segments_ransac.keys():
        normal = segment_models[i][:3]  
        
      
        x = abs(normal[0])  
        y = abs(normal[1]) 
        
        if x > 0.8 and y < 0.1:  
            
            x_candidates.append(i)
        elif y > 0.8 and x < 0.1:
           
            y_candidates.append(i)
        
    
    return x_candidates, y_candidates

In [89]:
x_candidates, y_candidates = filter_wall_elements_by_normal(segments_ransac, segment_models)

In [90]:
x_candidates, y_candidates

([], [0, 1, 3])

In [87]:
o3d.visualization.draw_geometries([segments_ransac[i] for i in y_candidates])

In [46]:
segments_ransac[3]

PointCloud with 1862 points.

In [205]:
def compute_bbox(segment_points):
    """
    Compute axis-aligned bounding box
    """
    min_coords = np.min(segment_points, axis=0)
    max_coords = np.max(segment_points, axis=0)
    
    dimensions = max_coords - min_coords
    
    return {
        'min': min_coords,
        'max': max_coords,
        'dimensions': dimensions,
        # 'volume': np.prod(dimensions),
        'center': (min_coords + max_coords) / 2
    }

In [206]:
np.array(segments_ransac[1].points)

array([[-18.735,  36.475,   2.562],
       [-18.648,  36.476,   2.571],
       [-18.827,  36.475,   2.554],
       ...,
       [-15.846,  36.465,   2.614],
       [-15.846,  36.475,   2.637],
       [-15.856,  36.534,   2.582]], shape=(4473, 3))

In [207]:
compute_bbox(np.array(segments_ransac[1].points))

{'min': array([-20.602,  36.406,   2.274]),
 'max': array([-15.758,  36.653,   2.671]),
 'dimensions': array([4.844, 0.247, 0.397]),
 'center': array([-18.18  ,  36.5295,   2.4725])}

In [208]:
compute_bbox(np.array(pcd.points))

{'min': array([-21.127,  33.715,   0.045]),
 'max': array([-15.231,  36.707,   3.371]),
 'dimensions': array([5.896, 2.992, 3.326]),
 'center': array([-18.179,  35.211,   1.708])}